In [1]:
!pip install nltk

In [2]:
!python -m pip install --upgrade pip


In [3]:
!pip install flask

In [4]:
!pip install flask fuzzywuzzy pandas

In [5]:
!pip install flask pandas nltk fuzzywuzzy openpyxl

In [6]:
!pip install pandas nltk fuzzywuzzy Flask

In [7]:
!pip install fastapi uvicorn nest-asyncio pandas nltk fuzzywuzzy

In [8]:
!pip install python-multipart

In [9]:
!pip install fastapi uvicorn nest-asyncio pandas nltk fuzzywuzzy

In [10]:
!python -m pip install --upgrade pip setuptools wheel

In [11]:
!pip install spacy --no-deps

In [12]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ----- ---------------------------------- 1.8/12.8 MB 10.1 MB/s eta 0:00:02
     ------------- -------------------------- 4.2/12.8 MB 10.5 MB/s eta 0:00:01
     -------------------- ------------------- 6.6/12.8 MB 10.6 MB/s eta 0:00:01
     --------------------------- ------------ 8.7/12.8 MB 10.5 MB/s eta 0:00:01
     -------------------------------- ------ 10.7/12.8 MB 10.3 MB/s eta 0:00:01
     --------------------------------------  12.6/12.8 MB 10.4 MB/s eta 0:00:01
     ---------------------------------------- 12.8/12.8 MB 9.8 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
from flask import Flask, render_template_string, request
import pandas as pd
from fuzzywuzzy import fuzz, process
import re
import webbrowser
from threading import Timer

# Initialize Flask app
app = Flask(__name__)

# Example dataset for diseases and symptoms
disease_bodyfn_df = pd.DataFrame({
    'Disease': ['asthma', 'cough'],
    'Symptoms': ['shortness of breath, wheezing, coughing, chest tightness', 'dry throat']
})

bodyfn_assistive_df = pd.DataFrame({
    'Symptoms': ['shortness of breath', 'wheezing', 'coughing', 'chest tightness', 'dry throat'],
    'Assistive Technology': ['oxygen therapy devices', 'breathing aids', 'humidifiers', 'No assistive technology found', 'humidifiers']
})

bodyfn_home_icf_df = pd.DataFrame({
    'Symptoms': ['shortness of breath', 'wheezing', 'coughing', 'chest tightness', 'dry throat'],
    'Home functions': ['open spaces for better airflow', 'soft padding in key resting areas', 
                       'air purifiers and improved ventilation', 'No home adaptations found', 'improved ventilation']
})

# Helper functions
def preprocess_text(text):
    """Normalize text input for easier matching."""
    return re.sub(r'[^a-zA-Z0-9\s]', '', text.strip().lower())

def find_closest_match(input_text, text_list, threshold=80):
    """Find the closest match using fuzzy matching."""
    match = process.extractOne(input_text, text_list, scorer=fuzz.partial_ratio)
    return match[0] if match and match[1] >= threshold else None

def get_details(input_text):
    """Retrieve disease and symptom details with assistive tech and home adaptation recommendations."""
    input_text = preprocess_text(input_text)
    disease_list = disease_bodyfn_df['Disease'].tolist()
    symptom_list = bodyfn_assistive_df['Symptoms'].tolist()

    matched_disease = find_closest_match(input_text, disease_list)
    matched_symptom = find_closest_match(input_text, symptom_list)

    if matched_disease:
        # Retrieve disease details
        disease_info = disease_bodyfn_df[disease_bodyfn_df['Disease'] == matched_disease]
        symptoms = disease_info['Symptoms'].values[0].split(', ') if not disease_info.empty else []
    elif matched_symptom:
        # Treat as a symptom if no disease match
        symptoms = [matched_symptom]
    else:
        # No match found
        symptoms = [input_text]
    
    assistive_mapping = []
    home_mapping = []

    for symptom in symptoms:
        symptom = preprocess_text(symptom)
        assistive_tech = find_closest_match(symptom, bodyfn_assistive_df['Symptoms'].tolist())
        home_adapt = find_closest_match(symptom, bodyfn_home_icf_df['Symptoms'].tolist())

        assistive_mapping.append((symptom, bodyfn_assistive_df.loc[
            bodyfn_assistive_df['Symptoms'] == assistive_tech, 'Assistive Technology'
        ].values[0] if assistive_tech else "No assistive technology found"))

        home_mapping.append((symptom, bodyfn_home_icf_df.loc[
            bodyfn_home_icf_df['Symptoms'] == home_adapt, 'Home functions'
        ].values[0] if home_adapt else "No home adaptations found"))
    
    return matched_disease, symptoms, assistive_mapping, home_mapping

# HTML Templates
disease_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <title>Disease and Symptom Details</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            background: linear-gradient(135deg, #e8f5e9, #a5d6a7);
            color: #333;
            margin: 0;
            padding: 0;
        }
        .navbar {
            background: #2e7d32;
            padding: 15px 0;
            display: flex;
            justify-content: center;
            text-align: center;
        }
        .navbar a {
            color: white;
            text-decoration: none;
            font-weight: bold;
            margin: 0 15px;
            font-size: 18px;
        }
        .navbar a:hover {
            text-decoration: underline;
            color: #c5e1a5;
        }
        .container {
            max-width: 600px;
            margin: 50px auto;
            padding: 30px;
            background: white;
            border-radius: 10px;
            box-shadow: 0 4px 10px rgba(0, 0, 0, 0.2);
            text-align: center;
        }
        input[type="text"] {
            width: 100%;
            padding: 15px;
            margin-bottom: 15px;
            font-size: 18px;
            border: 1px solid #ccc;
            border-radius: 5px;
        }
        button {
            padding: 10px 20px;
            font-size: 18px;
            background: #2e7d32;
            color: white;
            border: none;
            border-radius: 5px;
            cursor: pointer;
        }
        button:hover {
            background: #1b5e20;
        }
        h1 {
            color: #2e7d32;
        }
        ul {
            text-align: left;
        }
    </style>
</head>
<body>
    <div class="navbar">
        <a href="/">Home</a>
        <a href="/disease">Enter Disease Details</a>
        <a href="/symptom">Enter Symptom Details</a>
    </div>
    <div class="container">
        <h1>Enter Disease or Symptom Details</h1>
        <form method="post">
            <input type="text" name="disease_name" placeholder="Enter Disease or Symptom">
            <button type="submit">Search</button>
        </form>
        {% if output %}
        <div style="text-align:left;">
            {{ output|safe }}
        </div>
        {% endif %}
    </div>
</body>
</html>
"""

@app.route('/disease', methods=['GET', 'POST'])
def disease_search():
    if request.method == 'POST':
        input_text = request.form.get('disease_name')
        matched_disease, symptoms, assistive_mapping, home_mapping = get_details(input_text)

        # Generate conversational output
        output = f"<p>Hello! Based on your input, it seems you are referring to <strong>{matched_disease.capitalize() if matched_disease else 'a symptom'}</strong>.</p>"
        if symptoms:
            output += f"<p>This typically presents the following symptoms: <strong>{', '.join(symptoms)}</strong>.</p>"
        if assistive_mapping:
            output += "<p>To help manage these symptoms, you might consider the following assistive technologies:</p><ul>"
            for symptom, tech in zip(symptoms, assistive_mapping):
                output += f"<li>For <strong>{symptom.capitalize()}</strong>, consider using <em>{tech}</em>.</li>"
            output += "</ul>"
        if home_mapping:
            output += "<p>Additionally, here are some home adaptations that could be helpful:</p><ul>"
            for symptom, adapt in zip(symptoms, home_mapping):
                output += f"<li>For <strong>{symptom.capitalize()}</strong>, consider: <em>{adapt}</em>.</li>"
            output += "</ul>"

        return render_template_string(disease_template, output=output)

    return render_template_string(disease_template, output="")

def open_browser():
    """Automatically open the browser after the server starts."""
    webbrowser.open_new("http://127.0.0.1:5016/disease")

if __name__ == "__main__":
    Timer(1, open_browser).start()  # Automatically open the browser after 1 second
    app.run(port=5016, debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5016 (Press CTRL+C to quit)
127.0.0.1 - - [17/Dec/2024 12:43:33] "GET /disease HTTP/1.1" 200 -
127.0.0.1 - - [17/Dec/2024 12:43:34] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [17/Dec/2024 12:43:43] "POST /disease HTTP/1.1" 200 -
